# Setup

In [1]:
!pip install pyspark

In [1]:
from pyspark.ml.classification import LinearSVC, GBTClassifier, LogisticRegression, LinearSVCModel
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.sql import types as t
import numpy as np

In [2]:
spark = (
    SparkSession.builder.config("spark.executor.memory", "4g")
    .config("spark.executor.cores", "2")
    .config("spark.cores.max", "2")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

# Read the dataset(s)

Make sure to upload the train datasets generated by the `make-dataset.sh` script before proceeding.
Either train using X4 or X3,X2

In [ ]:
!unzip X4.zip

Archive:  X4.zip
replace X4.parquet/.part-00000-34b7377e-848c-4866-bfc8-33e858f2cd38-c000.snappy.parquet.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
df = spark.read.parquet('./X4.parquet')

In [3]:
df1 = spark.read.parquet('./X2.parquet')
df2 = spark.read.parquet('./X3.parquet')
df = df1.unionAll(df2)

In [ ]:
df.toPandas()

In [4]:
def stratified_split_train_test(df, frac, label,seed=42):
    fractions = df.select(label).distinct().withColumn("fraction", f.lit(frac)).rdd.collectAsMap()
    df_frac = df.stat.sampleBy(label, fractions, seed)
    df_remaining = df.exceptAll(df_frac)
    return df_frac, df_remaining

train_set, test_set = stratified_split_train_test(df, 0.8, 'label')

def with_weights(df, column="label"):
    w_zero = 1 / df.filter(f.col(column) == 0).count()
    w_one = 1 / df.filter(f.col(column) == 1).count()
    return df.withColumn("weight", f.when(f.col(column) == 0, w_zero).otherwise(w_one))

train_set = with_weights(train_set)


# Train the model gbt

In [7]:
model = GBTClassifier(weightCol='weight', maxIter=200, predictionCol='prediction')
model.setLossType('logistic')
model.setMaxDepth(8)
model.setMaxBins(28)
estimator = model.fit(train_set)

In [10]:
!rm -r model-notebooks
estimator.save("model-notebooks")

rm: cannot remove 'model-notebooks': No such file or directory


In [41]:
model = LinearSVC(weightCol='weight', maxIter=1000000, predictionCol='prediction')
paramGrid = ParamGridBuilder()\
    .addGrid(model.threshold, [1.0,1.1,1.2,1.3,1.4,1.5])\
    .build()

cvs = CrossValidator(estimator=model,
                           estimatorParamMaps=paramGrid,
                     evaluator=BinaryClassificationEvaluator(weightCol='weight', metricName='areaUnderPR'),
                     numFolds=4,
                    )
model.setThreshold(1.00001)
estimator = model.fit(train_set)

In [9]:
!rm -r model-products
estimator.save("model-products")

rm: cannot remove 'model-products': No such file or directory


# Prediction and evaluation

In [8]:
prediction = estimator.transform(test_set).drop('rawPrediction').drop('probability')

tp = prediction.filter("label==1 AND prediction==1").count()
p = tp / prediction.filter('prediction==1').count()
r = tp / prediction.filter('label == 1').count()
f1 = 2 * p * r / (p + r)
print("precision", p)
print("recall", r)
print("F1 score: ", f1)

precision 0.8635451505016722
recall 0.989272030651341
F1 score:  0.9221428571428572


In [11]:
!zip -r -9 models.zip model-notebooks

  adding: model-notebooks/ (stored 0%)
  adding: model-notebooks/data/ (stored 0%)
  adding: model-notebooks/data/part-00003-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet (deflated 8%)
  adding: model-notebooks/data/part-00002-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet (deflated 8%)
  adding: model-notebooks/data/.part-00000-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet.crc (stored 0%)
  adding: model-notebooks/data/.part-00002-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet.crc (stored 0%)
  adding: model-notebooks/data/._SUCCESS.crc (stored 0%)
  adding: model-notebooks/data/.part-00001-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet.crc (stored 0%)
  adding: model-notebooks/data/part-00001-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet (deflated 7%)
  adding: model-notebooks/data/.part-00003-e013ceb3-8ef6-4849-b4fb-f50f8356022e-c000.snappy.parquet.crc (stored 0%)
  adding: model-notebooks/data/part-00000-e013ceb3-8ef6-4

In [46]:
!zip -r -9 models.zip model-products

updating: model-products/ (stored 0%)
updating: model-products/data/ (stored 0%)
updating: model-products/data/._SUCCESS.crc (stored 0%)
updating: model-products/data/_SUCCESS (stored 0%)
updating: model-products/metadata/ (stored 0%)
updating: model-products/metadata/.part-00000.crc (stored 0%)
updating: model-products/metadata/part-00000 (deflated 44%)
updating: model-products/metadata/._SUCCESS.crc (stored 0%)
updating: model-products/metadata/_SUCCESS (stored 0%)
  adding: model-products/data/part-00000-918217d8-7528-45f5-a751-2dcb8acfe31a-c000.snappy.parquet (deflated 48%)
  adding: model-products/data/.part-00000-918217d8-7528-45f5-a751-2dcb8acfe31a-c000.snappy.parquet.crc (stored 0%)
